In [ ]:
from nbdev import *
# default_exp deployment

In [ ]:
# export
import flask
from flask import Flask
import requests
import json
from flask import request, jsonify
from tempfile import mkdtemp
import os.path as path
from securereqnet.preprocessing import vectorize_sentences
import numpy as np
from waitress import serve

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Deployment 🚀
This endpoint is designed to work in tandem with TFX serving to present the model in an easily consumable REST API where users pass in sentences and receive True/False with respect to whether the issue is security related or not.  The endpoint requires a port and hostname to be set up on, and is ready to deploy using waitress.

In [ ]:
#exporti
# takes a list of sentences, vectorizes them and sends the result to the uri of the TFX serving endpoint
def __get_predictions(sentences, endpoint_uri):
    payload = {
            "instances": vectorize_sentences(sentences).tolist()
        }
        
    r = requests.post(endpoint_uri, json = payload)
    model_preds = json.loads(r.content.decode('utf-8'))

    
    preds = []

    # decode predictions
    for pred in model_preds['predictions']:
        preds.append(__decode(pred))

    output = {
        "predictions": preds
    }

    return output




In [ ]:
#exporti
# decodes the tensor output from the TFX endpoint to True/False values
def __decode(input):
    return float(input[0])>float(input[1])

## Flask Backend
Defines a factory for our app.  Serving is packaged nicely inside a serve method, and is ready to deploy with waitress.

Factory method for our application.  It returns an instance of our Flask application.  Default configuration has the TFX serving predict API on http://localhost:8503/v1/models/alpha:predict

In [ ]:
#export

def create_app(test_config=None):
    """
    Returns a Flask web application with the default configuration pinging a TFX serving instance
    on http://localhost:8503/v1/models/alpha:predict
    """
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        TFX_ENDPOINT='http://localhost:8503/v1/models/alpha:predict'
    )

    if test_config:
        app.config.from_mapping(test_config)

    # default route, we probably will get rid of this
    @app.route('/', methods=['GET'])
    def home():
        return '<h1>SecureReqNet</h1><p>Flask backend</p>'

    # alpha model
    @app.route('/models/alpha', methods=['POST'])
    def alpha():
        content = request.get_json()
        sentences = content['instances']
        return __get_predictions(sentences, app.config['TFX_ENDPOINT'])

    return app




Using waitress, starts up a production server on the input host and port

In [ ]:
#export

def serve(host, port):
    """
    Serves a waitress production server on the given host and post
    """
    serve(create_app(), host = host, port = port)

In [ ]:
#all_deployment
# The following are test cases for deployment

In [ ]:
import unittest.mock
from unittest import mock
from unittest.mock import patch
from unittest.mock import Mock
from securereqnet.deployment import create_app

In [ ]:
# Set up test client
app = create_app({'TESTING': True, "TFX_ENDPOINT": "MockMockMock"})
test_client = app.test_client()

In [ ]:
# This tests to see if our app successfully deploys with a homepage
# Succeeds if the response code for default path is 200
def test_app_homepage():
    response = test_client.get('/')
    assert response.status_code==200

test_app_homepage()

In [ ]:
#collapse_input
# This test checks if we are able to successfully post a sentence to the server and get a response back
# Mock the backend request to TFX serving
# Succeeds if data returns successfully in correct form
@patch('securereqnet.deployment.__get_predictions')
def test_get_prediction_single(mock_predictions):
    test_payload = {"instances": ["test test test test"]}
    expected_pred = {"predictions": [True]}
    mock_predictions.return_value = expected_pred
    response = test_client.post('/models/alpha', json=test_payload)
    r_data = json.loads(response.data.decode('utf-8'))
    assert r_data == expected_pred

test_get_prediction_single()

['test test test test']


In [ ]:
#collapse_input
# This test checks if we are able to successfully post multiple sentences to the server and get a response back
# Mock the backend request to TFX serving
# Succeeds if data returns successfully in correct form
@patch('securereqnet.deployment.__get_predictions')
def test_get_prediction_multi(mock_predictions):
    test_payload = {"instances": ["test test test test", "more testing", "super duper testing"]}
    expected_pred = {"predictions": [True, False, True]}
    mock_predictions.return_value = expected_pred
    response = test_client.post('/models/alpha', json=test_payload)
    r_data = json.loads(response.data.decode('utf-8'))
    assert r_data == expected_pred

test_get_prediction_multi()

['test test test test', 'more testing', 'super duper testing']


In [ ]:
#collapse_input
# This test checks if we are able to successfully post a blank to the server and get a response back
# Mock the backend request to TFX serving
# Succeeds if data returns with a blank result and no error
@patch('securereqnet.deployment.__get_predictions')
def test_get_prediction_blank(mock_predictions):
    test_payload = {"instances": []}
    expected_pred = {"predictions": []}
    mock_predictions.return_value = expected_pred
    response = test_client.post('/models/alpha', json=test_payload)
    r_data = json.loads(response.data.decode('utf-8'))
    assert r_data == expected_pred

test_get_prediction_blank()

[]


In [ ]:
# testing for backend services

In [ ]:
#collapse_input
# checks if we can send a blank input and get a blank output with no error
@patch('requests.post')
def test_get_predictions_blank(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": []}")
    mock_post.return_value = mock
    test_payload = []
    expected_pred = {"predictions": []}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_blank()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


UnboundLocalError: local variable 'inp' referenced before assignment

In [ ]:
#collapse_input
# checks to see if we can post a single value which will decode as false
@patch('requests.post')
def test_get_predictions_single_false(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[0, 1]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test"]
    expected_pred = {"predictions": [False]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_single_false()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#collapse_input
# checks to see if we can send a single value which will decode as true
@patch('requests.post')
def test_get_predictions_single_true(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[1, 0]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test"]
    expected_pred = {"predictions": [True]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_single_true()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#collapse_input
# checks to see if we can test a nonbinary prediction on a single input and get true
@patch('requests.post')
def test_get_predictions_single_true_nonbinary(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[0.9, 0.2]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test"]
    expected_pred = {"predictions": [True]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_single_true_nonbinary()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#collapse_input
# checks to see if we can test a nonbinary prediction on a single input and get false
@patch('requests.post')
def test_get_predictions_single_false_nonbinary(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[0.2, 0.9]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test"]
    expected_pred = {"predictions": [False]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_single_false_nonbinary()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#collapse_input
# tests if we can get multiple true predictions from strings
@patch('requests.post')
def test_get_predictions_multi_true(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[1, 0], [1, 0], [1, 0]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test", "test test test test", "test test test test"]
    expected_pred = {"predictions": [True, True, True]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_multi_true()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#collapse_input
# tests if we can get multiple false predictions from strings
@patch('requests.post')
def test_get_predictions_multi_false(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[0, 1], [0, 1], [0, 1]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test", "test test test test", "test test test test"]
    expected_pred = {"predictions": [False, False, False]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_multi_false()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#collapse_input
# tests if we can get multiple mixed predictions from strings
@patch('requests.post')
def test_get_predictions_multi_mixed(mock_post):
    mock = Mock()
    mock.content.decode = Mock(return_value="{\"predictions\": [[0, 1], [1, 0], [0, 1], [1, 0]]}")
    mock_post.return_value = mock
    test_payload = ["test test test test", "test test test test", "test test test test", "test test test test"]
    expected_pred = {"predictions": [False, True, False, True]}
    response = __get_predictions(test_payload, "")
    assert expected_pred==response
    
test_get_predictions_multi_mixed()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
